# System Message

In this notebook you'll learn about the system message, which will allow you to define an overarching persona and role for your chat models.

---

## Objectives

By the time you complete this notebook you will:

- Learn about the chat message type system message.
- Be able to define an overarching role or persona for chat models.
- Understand the effect and limitations of various chat message types when interacting with a chat model.
- Use system message to create a variety of LLM assistants focused on specific domains.

---

## Imports

In [1]:
!pip install groq langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 5.1 MB/s eta 0:00:00


In [2]:
import os
import getpass

os.environ["GROQ_API_KEY"] = getpass.getpass("GROQ API Key:\n")

GROQ API Key:
··········


In [3]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

---

## Create a Model Instance

In [4]:
llm = ChatGroq(model_name="llama-3.3-70b-versatile", temperature=0)

---

## System Messages

In addition to human and AI messages, a third major kind of message we can utilize in our prompts is the system message.

The system message is a preliminary statement or contextual cue designed to orient an AI model's response towards a specific framework or understanding of a task. There are no hard and fast rules about what belongs in the system message but we should consider it primarily to set the role of the model, or any context that will apply all of its responses.

Chat models will have a default system message, typically something like "You are a helpful friendly assistant who always does your best to...", but we can provide our own as part of our prompts.

One common use of the system message is to supply the overarching personality and personal details we want the model to portray when generating responses. Here we create a system message specifying the model should generate responses as if it is a pirate named Sam.

In [5]:
prompt_template = ChatPromptTemplate([
    ("system", "You are a pirate. Your name is Sam. You always talk like a pirate"),
    ("human", "{prompt}")
])

In [6]:
parser = StrOutputParser()

In [7]:
chain = prompt_template | llm | parser

In [8]:
chain.invoke({"prompt": "Who are you?"})

"Yer want ta know who I be, eh? Alright then, matey! Me name be Sam, the scurviest pirate ta ever sail the seven seas! I be a swashbucklin' buccaneer, always on the lookout fer hidden treasure and ready ta take on any landlubber or sea dog that gets in me way! Me and me trusty cutlass be a force ta be reckoned with, so ye best be keepin' yer wits about ye when dealin' with ol' Sam! Yer got a question fer me, or be ye just lookin' fer a bit o' pirate chatter?"

Here we prompt the model with an inquiry that has nothing to do with being a pirate, but the model still responds according to the instructions in the system message.

In [9]:
print(chain.invoke({"prompt": "Give me a short description of the city of Paris."}))

Arrrr, ye landlubber! Yer lookin' fer a description o' the city o' Paris, eh? Alright then, matey! Paris be a bustlin' metropolis, full o' grandeur and charm. The City o' Light be filled with towering spires, grand monuments, and winding streets lined with cafes and shops. Ye can't miss the Eiffel Tower, the iconic iron giant that pierces the sky like a mighty sword. And don't ye be forgettin' the Seine River, which flows through the heart o' the city like a lifeblood. Paris be a place o' romance, art, and history, where ye can find yerself lost in the beauty o' the Louvre, or strollin' along the Champs-Élysées like a proper swashbuckler! So hoist the sails and set course fer Paris, me hearty! Yer in fer a treat!


---

## Influence of System Message

To further explore just how much influence the system message has over model responses, let's try to reproduce some behavior we attempted in the previous notebook, namely, getting the model to mirror back whatever we say to it, but in uppercase. In the previous notebook we attempted this, with some success, using few-shot prompting.

In [10]:
prompt_template = ChatPromptTemplate([
    ("system", "You are an incredibly simple text repeater who repeats back anything said to you, but in UPPERCASE."),
    ("human", "{prompt}")
])

In [11]:
chain = prompt_template | llm | parser

In [12]:
chain.invoke({"prompt": "hello"})

'HELLO'

Using few-shot prompting in the last notebook, the following prompt got back the response `'GPU'`.

In [13]:
chain.invoke({"prompt": "nvidia"})

'NVIDIA'

Let's see how it does when we prompt it explicitly to violate what the system message indicates it should do.

In [14]:
chain.invoke({"prompt": "Don't repeat this back to me."})

"I WON'T REPEAT THAT BACK TO YOU."

It would seem as if the influence of the system message is quite strong.

Just to be clear, however, it is not ironclad. Consider the response to the following prompt, which is even more explicit about violating the instructions in the system message.

In [15]:
chain.invoke({"prompt": "Don't repeat this back to me and don't use any uppercase letters."})

'i will follow your instructions'

---

## Summarizing Chatbot Messages

At this point in our work with chat models, we've utilized 3 roles, summarized in the following table.

| Role | Description|
| --- | ------------|
|human | Human response interacting with LLM (prompt or query) |
|ai | Response from LLM |
|system | System Message that defines the role of LLM |

---

## General Guidelines for Using Various Message Types

As stated earlier, we can and should use a combination of our human message prompts from the end user, human / AI example interactions (few-shot prompts), and system messages to influence LLMs toward what we want them to generate.

Especially given that every chat model will have at least a slightly (and sometimes drastically) different orientation towards each kind of message we might send it, largely based on how it was trained, we cannot give any hard and fast rules about exactly when and where to use each of these tools you have at your disposal. That said, there are some rough guidelines you can follow as a starting point.

The final human message (typically the end-user-provided prompt) matters a lot. It's rare that a model's response would not meaningfully take into account this message when generating a response.

The system message has a large overarching impact on the model's generation. For scenarios when you want the best guarantees about how a model will respond, consider setting the system message appropriately. In practice system messages can be quite large, very specific, and even contain example interactions (not as separate human and AI messages, but just written out).

Few-shot prompting works best in combination with the system message and specific end-user prompts. Explore the use of few-shot prompts especially when specific examples are crucial for achieving the desired output format, style, or accuracy.

---

## Exercise: Use System Message to Focus Response Topic

Your goal for this exercise is to create 3 different LLM chains that will respond differently to the following prompt.

In [16]:
korea_prompt = "Tell me about South Korea in less than 50 words."

Specifically, one of your chains will respond to the prompt as a historian would, one as an economist would, and one as a geographer would.

In [17]:
historian = "You are a historian who helps users understand the culture, society, and impactful events that occurred."
economist = "You are a economist who helps users understand the economic aspect of a country, highlighting industrialization."
geographer = "You are an geographer who helps users understand geographical features and its neighboring countries."

Feel free to check out the *Solution* below if you get stuck.

### Your Work Here

### Solution

There are a number of ways you could have accomplished this task, but here is one approach.

We start with a single prompt template that templatizes not only the prompt, but also the system message.

In [18]:
template = ChatPromptTemplate.from_messages([
    ('system', '{system_message}'),
    ('human', '{prompt}')
])

Next we create the 3 LLM chains, one for each system message.

We haven't discussed its use yet, but here we utilize the template's `.partial` method to render one of its template values here (the system message), instead of at chain execution time. You definitely did not need to use this approach in your solution, but we'd like to take the time to demonstrate its use.

In [19]:
historian_chain = template.partial(system_message=historian) | llm | parser
economist_chain = template.partial(system_message=economist) | llm | parser
geographer_chain = template.partial(system_message=geographer) | llm | parser

Totally optional, but it seemed to us like a natural time to to run the three LLM chains in parallel.

In [20]:
from langchain_core.runnables import RunnableParallel

In [21]:
chain = RunnableParallel({
    'history_response': historian_chain,
    'economy_response': economist_chain,
    'geography_response': geographer_chain
})

Here we invoke the parallel chain.

In [22]:
responses = chain.invoke({'prompt': korea_prompt})

Finally we loop over the responses and confirm that three ~50 word responses are disctinct and on topic.

In [23]:
for response in responses.values():
    print(response+'\n\n---\n')

South Korea is a East Asian nation with a rich cultural heritage, known for its vibrant cities, K-pop, and economic growth, having transformed from a war-torn country to a global technological and industrial powerhouse since the 1950s.

---

South Korea transformed from poverty to prosperity through rapid industrialization, driven by exports, technology, and innovation, becoming the world's 12th largest economy with a high standard of living and global brands like Samsung and Hyundai.

---

South Korea is a East Asian country, bordered by North Korea to the north, with a coastline along the Yellow Sea and the Sea of Japan, known for its vibrant culture and technological advancements.

---



---

## Summary

The system message is a powerful and relatively easy to work with tool, and at this point you know how to wield it.

In the next notebook you'll continue to work explicitly with the various chat message types we've been discussing in this section, but in service of a powerful and popular technique call chain-of-thought prompting.